In [1]:
import requests

lat, lon = 40.7128, -74.0060  # Example: New York City coordinates

query = f"""
[out:json];
(
  node(around:50,{lat},{lon})["building"];
  way(around:50,{lat},{lon})["building"];
);
out body;
"""

url = "http://overpass-api.de/api/interpreter"
response = requests.get(url, params={"data": query})

data = response.json()
print(data)  # Returns building information around the coordinates


{'version': 0.6, 'generator': 'Overpass API 0.7.62.5 1bd436f1', 'osm3s': {'timestamp_osm_base': '2025-03-29T10:48:16Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'way', 'id': 575213527, 'nodes': [5519996274, 5519996275, 5519996276, 5519996277, 5519996278, 5519996279, 9368527083, 5519996280, 2242624721, 5519996282, 5519996283, 5519996284, 5519996285, 5519996286, 5519996287, 9368823842, 5519996288, 5519996289, 5519996274], 'tags': {'addr:city': 'New York', 'addr:housename': 'New York City Hall', 'addr:housenumber': '260', 'addr:postcode': '10000', 'addr:state': 'NY', 'addr:street': 'Broadway', 'amenity': 'townhall', 'architect': 'Joseph Francois Mangin and John McComb, Jr.', 'building': 'public', 'building:levels': '4', 'building:part': 'yes', 'height': '36.6', 'level': '2', 'name': 'New York City Hall', 'name:el': 'Δημαρχείο Νέας Υόρκης', 'name:hi': 'न्यूयॉर्क सिटी हॉल', 'name:hr': 'Gra

In [6]:
import requests

def get_land_use(lat, lon):
    query = f"""
    [out:json];
    (
      node(around:50,{lat},{lon})["building"];
      way(around:50,{lat},{lon})["highway"];
      way(around:50,{lat},{lon})["landuse"];
      way(around:50,{lat},{lon})["natural"];
      
    );
    out body;
    """
    
    url = "http://overpass-api.de/api/interpreter"
    response = requests.get(url, params={"data": query})
    
    if response.status_code == 200:
        data = response.json()
        if data["elements"]:
            return "Building Found"
        else:
            return "No Building Here"
    else:
        return "Error Fetching Data"

# Example: Get land use info for a location
lat, lon = 28.6139, 77.2090  # Example: Delhi, India
print(get_land_use(lat, lon))


Building Found


In [11]:
import requests

# Function to fetch land use data from OpenStreetMap
def get_osm_data(lat, lon):
    query = f"""
    [out:json];
    (
      node(around:50,{lat},{lon})["building"];
      way(around:50,{lat},{lon})["building"];
      way(around:50,{lat},{lon})["highway"];
      way(around:50,{lat},{lon})["landuse"];
      way(around:50,{lat},{lon})["natural"];
    );
    out body;
    """
    
    url = "http://overpass-api.de/api/interpreter"
    response = requests.get(url, params={"data": query})
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to categorize land use based on OSM tags
def categorize_land_use(data):
    categories = {
        "building": "Building",
        "highway": "Road",
        "landuse": {
            "residential": "Residential Area",
            "commercial": "Commercial Area",
            "industrial": "Industrial Zone",
            "retail": "Shopping Area",
            "forest": "Forest",
            "farmland": "Farmland",
            "grass": "Grassland",
            "park": "Park"
        },
        "natural": {
            "water": "Water Body",
            "forest": "Forest",
            "wood": "Wooded Area"
        }
    }

    land_types = set()  # To avoid duplicate results
    if data and "elements" in data:
        for element in data["elements"]:
            tags = element.get("tags", {})
            for tag, value in tags.items():
                if tag in categories:
                    if isinstance(categories[tag], dict):  # Handle nested categories
                        land_types.add(categories[tag].get(value, value))
                    else:
                        land_types.add(categories[tag])

    return land_types if land_types else {"Unknown Land Use"}

# Main function to get land
def get_land_type(lat, lon):
    data = get_osm_data(lat, lon)
    land_types = categorize_land_use(data)
    return land_types

# Example: Check land use at a given location
lat, lon = 28.616355680702913, 77.21516220278437  # Example: Delhi, India
result = get_land_type(lat, lon)
print(f"Land Use at ({lat}, {lon}): {', '.join(result)}")

Land Use at (28.616355680702913, 77.21516220278437): Building, Road
